In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process.kernels import RBF
from scipy import stats

In [ ]:
def make_mu(n):
    mu = 0.5
    return mu*np.ones(shape=(n,1))

def make_kernel(a, b=None):
    std_dev = 0.15
    var = std_dev**2
    k = RBF()
    if b is None:
        return var*(k(a) + np.eye(a.shape[0])*1e-6)
    else:
        return var*k(a, b)

In [ ]:
def conditional(x, x_given=None, y_given=None):
    if x_given is None or y_given is None:
        return (make_mu(x.shape[0]), make_kernel(x))
    else:
        s_new_given = make_kernel(x, x_given)
        s_given = make_kernel(x_given)
        s_new = make_kernel(x)
        mu_given = make_mu(y_given.shape[0])
        mu_new =  make_mu(x.shape[0])

        mu = mu_new + s_new_given.dot(np.linalg.inv(s_given)).dot(y_given-mu_given)
        sig = s_new - s_new_given.dot(np.linalg.inv(s_given)).dot(s_new_given.T)
        return (mu.squeeze(), sig.squeeze())
    

In [ ]:
def plot_conditional(x, given=None, n_iters=10, n_priors=100, savefig=False):
    
    i = 0
    
    while i < n_iters:

        if given is None:
            given = []
            mu_cond, sig_cond = conditional(x)
            f_posterior = np.random.multivariate_normal(mu_cond.squeeze(), sig_cond, n_priors).T
        else:
            temp = np.array(given)
            x_given = temp[:, 0].reshape(-1,1)
            y_given = temp[:, 1].reshape(-1,1)
            mu_cond, sig_cond = conditional(x, x_given=x_given, y_given=y_given)
            f_posterior = np.random.multivariate_normal(mu_cond, sig_cond, n_priors).T

        f, axs = plt.subplots(2, 1, figsize=(10,10))

        axs[0].plot(x, f_posterior, "lightblue")
        axs[0].plot(x, mu_cond, "r--", linewidth=2)

        axs[0].set_title("GP")
        axs[0].get_xaxis().set_ticks([])
        axs[0].set_xlabel("Weight vector")
        axs[0].set_ylabel("Utility")
        
        if len(given) > 0:
            delta = np.max(y_given) - mu_cond.mean()
            sig = np.sqrt(sig_cond.diagonal())
            z = delta / sig
            ei = delta*stats.norm.cdf(z) + sig*stats.norm.pdf(z)
            ei_max = ei.max()
            x_new = x[np.argmax(ei), 0]
            y_new = np.random.normal(loc=0.6, scale=0.15)
        else:
            delta = 0
            sig = np.sqrt(sig_cond.diagonal())
            z = delta / sig
            ei = delta*stats.norm.cdf(z) + sig*stats.norm.pdf(z)
            ei_max = ei.max()
            x_new = 0
            y_new = np.random.normal(loc=0.6, scale=0.15)
        
        given.append((x_new, y_new))

        axs[1].plot(x, ei)
        axs[1].plot(x_new, ei_max, "ro")
        axs[1].set_title("Acquisition Function")
        axs[1].get_xaxis().set_ticks([])
        axs[1].set_xlabel("Weight vector")
        axs[1].set_ylabel("Expected Improvement")

        plt.show()

        if savefig: f.savefig("GP{}.png".format(i+1))

        i = i + 1

In [ ]:
x = np.linspace(-5, 5, 1000).reshape(-1,1)
plot_conditional(x)